In [1]:
import sys
if not 'Informer2020' in sys.path:
    sys.path += ['Informer2020'] 

In [2]:
#!git clone https://github.com/zhouhaoyi/Informer2020.git

In [3]:
from utils.tools import dotdict

from exp.exp_informer import Exp_Informer
import torch

In [4]:
args = dotdict()

args.model = 'informer' # model of experiment, options: [informer, informerstack, informerlight(TBD)]

In [5]:
args = dotdict()

args.model = 'informer' # model of experiment, options: [informer, informerstack, informerlight(TBD)]

args.data = 'ETTh1' # data
args.root_path = './ETDataset/ETT-small/' # root path of data file
args.data_path = 'ETTh1.csv' # data file
args.features = 'M' # forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate
args.target = 'OT' # target feature in S or MS task
args.freq = 'm' # freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h
args.checkpoints = './informer_checkpoints' # location of model checkpoints

args.seq_len = 36   # use last 3 years as input history
args.label_len = 12 # use last year to warm up decoder
args.pred_len = 24  # predict next 2 years (24 months)

args.enc_in = 7 # encoder input size
args.dec_in = 7 # decoder input size
args.c_out = 7 # output size
args.factor = 5 # probsparse attn factor
args.d_model = 512 # dimension of model
args.n_heads = 8 # num of heads
args.e_layers = 2 # num of encoder layers
args.d_layers = 1 # num of decoder layers
args.d_ff = 2048 # dimension of fcn in model
args.dropout = 0.05 # dropout
args.attn = 'prob' # attention used in encoder, options:[prob, full]
args.embed = 'timeF' # time features encoding, options:[timeF, fixed, learned]
args.activation = 'gelu' # activation
args.distil = True # whether to use distilling in encoder
args.output_attention = False # whether to output attention in ecoder
args.mix = True
args.padding = 0
args.freq = 'm'

args.batch_size = 32 
args.learning_rate = 0.0001
args.loss = 'mse'
args.lradj = 'type1'
args.use_amp = False # whether to use automatic mixed precision training

args.num_workers = 0
args.itr = 1
args.train_epochs = 6
args.patience = 3
args.des = 'exp'

args.use_gpu = True if torch.cuda.is_available() else False
args.gpu = 0

args.use_multi_gpu = False
args.devices = '0,1,2,3'

In [6]:
args.use_gpu = True if torch.cuda.is_available() and args.use_gpu else False

if args.use_gpu and args.use_multi_gpu:
    args.devices = args.devices.replace(' ','')
    device_ids = args.devices.split(',')
    args.device_ids = [int(id_) for id_ in device_ids]
    args.gpu = args.device_ids[0]

In [7]:
# Set augments by using data name
data_parser = {
    'ETTh1':{'data':'ETTh1.csv','T':'OT','M':[7,7,7],'S':[1,1,1],'MS':[7,7,1]},
    'ETTh2':{'data':'ETTh2.csv','T':'OT','M':[7,7,7],'S':[1,1,1],'MS':[7,7,1]},
    'ETTm1':{'data':'ETTm1.csv','T':'OT','M':[7,7,7],'S':[1,1,1],'MS':[7,7,1]},
    'ETTm2':{'data':'ETTm2.csv','T':'OT','M':[7,7,7],'S':[1,1,1],'MS':[7,7,1]},
}
if args.data in data_parser.keys():
    data_info = data_parser[args.data]
    args.data_path = data_info['data']
    args.target = data_info['T']
    args.enc_in, args.dec_in, args.c_out = data_info[args.features]

In [8]:
args.detail_freq = args.freq
args.freq = args.freq[-1:]

In [9]:
print('Args in experiment:')
print(args)

Args in experiment:
{'model': 'informer', 'data': 'ETTh1', 'root_path': './ETDataset/ETT-small/', 'data_path': 'ETTh1.csv', 'features': 'M', 'target': 'OT', 'freq': 'm', 'checkpoints': './informer_checkpoints', 'seq_len': 36, 'label_len': 12, 'pred_len': 24, 'enc_in': 7, 'dec_in': 7, 'c_out': 7, 'factor': 5, 'd_model': 512, 'n_heads': 8, 'e_layers': 2, 'd_layers': 1, 'd_ff': 2048, 'dropout': 0.05, 'attn': 'prob', 'embed': 'timeF', 'activation': 'gelu', 'distil': True, 'output_attention': False, 'mix': True, 'padding': 0, 'batch_size': 32, 'learning_rate': 0.0001, 'loss': 'mse', 'lradj': 'type1', 'use_amp': False, 'num_workers': 0, 'itr': 1, 'train_epochs': 6, 'patience': 3, 'des': 'exp', 'use_gpu': False, 'gpu': 0, 'use_multi_gpu': False, 'devices': '0,1,2,3', 'detail_freq': 'm'}


In [10]:
Exp = Exp_Informer

In [11]:
for ii in range(args.itr):
    # setting record of experiments
    setting = '{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_at{}_fc{}_eb{}_dt{}_mx{}_{}_{}'.format(args.model, args.data, args.features, 
                args.seq_len, args.label_len, args.pred_len,
                args.d_model, args.n_heads, args.e_layers, args.d_layers, args.d_ff, args.attn, args.factor, args.embed, args.distil, args.mix, args.des, ii)

    # set experiments
    exp = Exp(args)
    
    # train
    print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
    exp.train(setting)
    
    # test
    #print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
    #exp.test(setting)

    torch.cuda.empty_cache()

Use CPU
>>>>>>>start training : informer_ETTh1_ftM_sl36_ll12_pl24_dm512_nh8_el2_dl1_df2048_atprob_fc5_ebtimeF_dtTrue_mxTrue_exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 274


RuntimeError: Given groups=1, weight of size [512, 7, 3], expected input[32, 6, 38] to have 7 channels, but got 6 channels instead

In [ ]:
# size = [seq_len, label_len, pred_len]
train_dataset = Dataset_ETT_hour(
    root_path='./ETDataset/ETT-small/',
    flag='train',
    size=[96, 48, 24],  # seq_len=96, label_len=48, pred_len=24
    features='S',
    target='OT',
    scale=True,
    inverse=False,
    timeenc=1,
    freq='m',
    cols=None
)

print("Train dataset length:", len(train_dataset))
print("seq_len:", train_dataset.seq_len, "label_len:", train_dataset.label_len, "pred_len:", train_dataset.pred_len)

In [ ]:
val_dataset = Dataset_ETT_hour(
    root_path='./ETDataset/ETT-small/',
    flag='val',
    size=[96, 48, 24],
    features='S',
    target='OT',
    scale=True,
    inverse=False,
    timeenc=1,
    freq='m',
    cols=None
)

test_dataset = Dataset_ETT_hour(
    root_path='./ETDataset/ETT-small/',
    flag='test',
    size=[96, 48, 24],
    features='S',
    target='OT',
    scale=True,
    inverse=False,
    timeenc=1,
    freq='m',
    cols=None
)

print("Val dataset length:", len(val_dataset))
print("Test dataset length:", len(test_dataset))

In [ ]:
import pandas as pd

df = pd.read_csv('./ETDataset/ETT-small/ETTh1.csv')  # or whichever CSV is used for val/test
print("Total rows:", len(df))

In [ ]:
# Assuming you already created datasets
train_dataset = Dataset_ETT_hour(root_path='./ETDataset/ETT-small/', flag='train',
                                 size=[96,48,24], features='S', target='OT',
                                 scale=True, inverse=False, timeenc=1, freq='m', cols=None)
val_dataset   = Dataset_ETT_hour(root_path='./ETDataset/ETT-small/', flag='val',
                                 size=[96,48,24], features='S', target='OT',
                                 scale=True, inverse=False, timeenc=1, freq='m', cols=None)
test_dataset  = Dataset_ETT_hour(root_path='./ETDataset/ETT-small/', flag='test',
                                 size=[96,48,24], features='S', target='OT',
                                 scale=True, inverse=False, timeenc=1, freq='m', cols=None)

# Print their internal borders / sizes
print("Train dataset rows:", train_dataset.__len__())
print("Val dataset rows:", val_dataset.__len__())
print("Test dataset rows:", test_dataset.__len__())

In [ ]:
import pandas as pd
from data.data_loader import Dataset_ETT_hour

# -----------------------
# PARAMETERS
# -----------------------
root_path = r'./ETDataset/ETT-small/'  # folder containing CSV
data_file = 'ETTh1.csv'
target = 'OT'
features = 'S'
freq = 'm'  # monthly data

# Sequence parameters
seq_len = 15
label_len = 5
pred_len = 10
size = [seq_len, label_len, pred_len]

# -----------------------
# LOAD RAW DATA
# -----------------------
df = pd.read_csv(root_path + data_file)
total_rows = len(df)
print("Total rows in CSV:", total_rows)

# -----------------------
# CALCULATE SAFE SPLITS
# -----------------------
min_rows_per_split = seq_len + pred_len  # minimum required rows per dataset

# Ensure enough rows for train, val, test
train_rows = total_rows - 2 * min_rows_per_split
val_rows = test_rows = min_rows_per_split

print("Train rows:", train_rows)
print("Val rows:", val_rows)
print("Test rows:", test_rows)

# -----------------------
# CREATE DATASET INSTANCES
# -----------------------
train_dataset = Dataset_ETT_hour(
    root_path=root_path,
    data_path=data_file,
    flag='train',
    size=size,
    features=features,
    target=target,
    scale=True,
    inverse=False,
    timeenc=1,
    freq=freq,
    cols=None,
    custom_start_end=(0, train_rows)
)

val_dataset = Dataset_ETT_hour(
    root_path=root_path,
    data_path=data_file,
    flag='val',
    size=size,
    features=features,
    target=target,
    scale=True,
    inverse=False,
    timeenc=1,
    freq=freq,
    cols=None,
    custom_start_end=(train_rows, train_rows + val_rows)
)

test_dataset = Dataset_ETT_hour(
    root_path=root_path,
    data_path=data_file,
    flag='test',
    size=size,
    features=features,
    target=target,
    scale=True,
    inverse=False,
    timeenc=1,
    freq=freq,
    cols=None,
    custom_start_end=(train_rows + val_rows, total_rows)
)

# -----------------------
# CHECK DATASET LENGTHS
# -----------------------
print("Train dataset length:", len(train_dataset))
print("Val dataset length:", len(val_dataset))
print("Test dataset length:", len(test_dataset))

In [ ]:
import os

# set saved model path
setting = 'informer_ETTh1_ftM_sl96_ll48_pl24_dm512_nh8_el2_dl1_df2048_atprob_fc5_ebtimeF_dtTrue_mxTrue_exp_0'
# path = os.path.join(args.checkpoints,setting,'checkpoint.pth')

In [ ]:
# If you already have a trained model, you can set the arguments and model path, then initialize a Experiment and use it to predict
# Prediction is a sequence which is adjacent to the last date of the data, and does not exist in the data
# If you want to get more information about prediction, you can refer to code `exp/exp_informer.py function predict()` and `data/data_loader.py class Dataset_Pred`

exp = Exp(args)

exp.predict(setting, True)